## Story Generator Local Server

[Google Notebook](https://colab.research.google.com/drive/1uNygzDR4hISwLOgmDS31hRHfr6KAF7Ib?usp=sharing#scrollTo=unique_id_here)

In [ ]:
!pip install -q transformers==4.35.2 accelerate bitsandbytes==0.41.1 huggingface_hub google-generativeai==0.3.1
!pip install -q Flask==2.3.2 Pillow==9.5.0 requests==2.30.0 flask-ngrok Flask-CORS==3.0.10 pyngrok
!pip install Flask-Cors
import os
from google.colab import userdata


# Use environment variables
from huggingface_hub import login
login(userdata.get('HUGGINGFACE_TOKEN'))

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

from IPython.display import clear_output
clear_output(wait=False)

print("Dependencies installed successfully!")

Dependencies installed successfully!


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import base64
from PIL import Image
import io
import google.generativeai as genai
import os
from google.colab import userdata

# Configure Gemini API
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')

# Configure ngrok
ngrok.set_auth_token(userdata.get('NGROK_AUTH_TOKEN'))

app = Flask(__name__)
CORS(app)

@app.route('/generate_story', methods=['POST'])
def generate_story():
    try:
        data = request.json
        if not data:
            return jsonify({'success': False, 'error': 'No data received'}), 400

        image_data = data.get('image', '')
        genre = data.get('genre', 'fantasy').lower()
        word_count = data.get('length', 200)  # Default to 200 words if not specified

        if image_data:
            try:
                # Process image
                image_bytes = base64.b64decode(image_data)
                image = Image.open(io.BytesIO(image_bytes))

                # Convert to RGB if needed
                if image.mode in ('RGBA', 'LA'):
                    background = Image.new('RGB', image.size, (255, 255, 255))
                    background.paste(image, mask=image.split()[-1])
                    image = background
                elif image.mode != 'RGB':
                    image = image.convert('RGB')

                # Resize if needed
                max_size = (1024, 1024)
                if image.size[0] > max_size[0] or image.size[1] > max_size[1]:
                    image.thumbnail(max_size, Image.LANCZOS)

                # Create model for image analysis
                vision_model = genai.GenerativeModel('gemini-1.5-pro')

                # Generate image prompt
                image_prompt = "Describe the objects and scene in this image."
                image_response = vision_model.generate_content([image_prompt, image])
                image_description = image_response.text
            except Exception as img_error:
                return jsonify({'success': False, 'error': f'Image processing error: {str(img_error)}'}), 400
        else:
            image_description = ""

        # Define age-appropriate themes for each genre
        genre_themes = {
            'fantasy': 'magical adventures, friendly creatures, and noble quests',
            'adventure': 'exploration, discovery, and overcoming challenges',
            'romance': 'friendship, kindness, and family relationships',
            'horror': 'mild mystery, spooky (but not scary) situations, and courage',
            'mystery': 'solving puzzles, helping others, and uncovering secrets',
            'moral story': 'learning life lessons, making good choices, and personal growth'
        }

        # Craft a more controlled prompt
        prompt = f"""Write a family-friendly {genre} story for children about {genre_themes.get(genre, 'adventure and discovery')}.
        Requirements:
        - Exactly {word_count} words (important!)
        - Age-appropriate content (suitable for ages 8-12)
        - No violence, scary elements, or adult themes
        - Focus on positive messages and character growth
        - Include descriptive but simple language
        - Clear beginning, middle, and end structure
        - Based on the following image description: {image_description}

        Create an engaging story that teaches good values while entertaining young readers."""

        # Configure safety settings
        safety_settings = [
            {
                "category": "HARM_CATEGORY_DANGEROUS",
                "threshold": "BLOCK_MEDIUM_AND_ABOVE"
            },
            {
                "category": "HARM_CATEGORY_HARASSMENT",
                "threshold": "BLOCK_MEDIUM_AND_ABOVE"
            },
            {
                "category": "HARM_CATEGORY_HATE_SPEECH",
                "threshold": "BLOCK_MEDIUM_AND_ABOVE"
            },
            {
                "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
                "threshold": "BLOCK_MEDIUM_AND_ABOVE"
            },
            {
                "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                "threshold": "BLOCK_MEDIUM_AND_ABOVE"
            },
        ]

        # Generate story using Gemini with safety settings
        response = model.generate_content(
            prompt,
            safety_settings=safety_settings,
            generation_config={
                "temperature": 0.7,
                "top_p": 0.8,
                "top_k": 40
            }
        )

        if response.text:
            return jsonify({'success': True, 'story': response.text})
        else:
            return jsonify({
                'success': False,
                'error': 'Story generation failed. Please try again with different parameters.'
            }), 500

    except Exception as e:
        print(f"Server error: {str(e)}")
        return jsonify({
            'success': False,
            'error': f'Server error: {str(e)}'
        }), 500

if __name__ == '__main__':
    try:
        # Kill any existing ngrok processes
        !pkill ngrok
        
        # Setup ngrok
        public_url = ngrok.connect(5000)
        print(f'Server running at: {public_url}')
        
        # Start Flask app
        app.run(port=5000)
    except Exception as e:
        print(f"Error starting server: {str(e)}")
        print("You may need to restart the Colab runtime if ngrok is already running")


t=2024-11-15T16:32:03+0530 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=C:\\Users\\Shravan\\AppData\\Local/ngrok/ngrok.yml legacy_path=C:\\Users\\Shravan\\.ngrok2\\ngrok.yml


Starting server...

Ngrok tunnel established at: NgrokTunnel: "https://c271-2409-40f3-101b-f886-7095-d6fa-4e8b-6f3e.ngrok-free.app" -> "http://localhost:5000"
Story generation endpoint: NgrokTunnel: "https://c271-2409-40f3-101b-f886-7095-d6fa-4e8b-6f3e.ngrok-free.app" -> "http://localhost:5000"/generate_story

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


DEBUG: Image mode: RGB


127.0.0.1 - - [15/Nov/2024 16:34:37] "POST /generate_story HTTP/1.1" 400 -


DEBUG: Image processing error: 404 Gemini 1.0 Pro Vision has been deprecated on July 12, 2024. Consider switching to different model, for example gemini-1.5-flash.
